In [28]:
import pandas as pd
import numpy as np
import re
import time

In [2]:
taxi_df = pd.read_csv("../../data/taxi/taxi_data.csv")
bus_df = pd.read_csv("../../data/taxi/bus_stations.csv")
subway_df = pd.read_csv("../../data/taxi/subway_stations.csv")

In [3]:
def parseCoords(regex, x):
    arr = regex.match(x)
    return (float(arr.group(2)), float(arr.group(1)))

In [4]:
regex = re.compile("[ ]*POINT \(([-.0-9]+) ([-.0-9]+)\)")
subway_df['coords'] = subway_df.apply(lambda x : parseCoords(regex, x['the_geom']), axis=1)
bus_df['coords'] = list(zip(bus_df['LATITUDE'], bus_df['LONGITUDE']))

In [5]:
subway_df['type'] = 'subway'
bus_df['type'] = 'bus'

In [7]:
bus_df['location'] = bus_df['Street'] + ' AT BETWEEN ' + bus_df['AT_BETWEEN']
bus_df_stripped = bus_df.filter(['location', 'coords', 'type'], axis=1)
subway_df['location'] = subway_df['NAME']
subway_df_stripped = subway_df.filter(['location', 'coords', 'type'], axis=1)
oepnv_locations = pd.concat([subway_df_stripped, bus_df_stripped], ignore_index=True, sort=False)

In [8]:
oepnv_locations

,location,coords,type
0,Astor Pl,"(40.73005400028978, -73.99106999861966)",subway
1,Canal St,"(40.71880300107709, -74.00019299927328)",subway
2,50th St,"(40.76172799961419, -73.98384899986625)",subway
3,Bergen St,"(40.68086213682956, -73.97499915116808)",subway
4,Pennsylvania Ave,"(40.66471445143568, -73.89488591154061)",subway
5,238th St,"(40.88466700064975, -73.90087000018522)",subway
6,Cathedral Pkwy (110th St),"(40.800581558114956, -73.95806670661364)",subway
7,Kingston - Throop Aves,"(40.67991899941601, -73.94085899871263)",subway
8,65th St,"(40.74971952935675, -73.8987883783301)",subway
9,36th St,"(40.75196004401078, -73.92901818461539)",subway


In [9]:
taxi_df_with_location = taxi_df[taxi_df['Pickup_longitude'].notnull()][taxi_df['Pickup_latitude'].notnull()][taxi_df['Dropoff_longitude'].notnull()][taxi_df['Dropoff_latitude'].notnull()]

C:\my-programmes\anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [12]:
taxi_df_with_location['trip_start_coords'] = list(zip(taxi_df_with_location['Pickup_latitude'], taxi_df_with_location['Pickup_longitude']))
taxi_df_with_location['trip_stop_coords'] = list(zip(taxi_df_with_location['Dropoff_latitude'], taxi_df_with_location['Dropoff_longitude']))

In [16]:
taxi_df_stripped = taxi_df_with_location.filter(['Passenger_count', 'Trip_distance', 'lpep_pickup_datetime', 'Lpep_dropoff_datetime', 'Total_amount', 'trip_start_coords', 'trip_stop_coords'], axis=1)
taxi_df_stripped.head(10)

,Passenger_count,Trip_distance,lpep_pickup_datetime,Lpep_dropoff_datetime,Total_amount,trip_start_coords,trip_stop_coords
5005541,1,1.46,01/01/2016 12:29:24 AM,01/01/2016 12:39:36 AM,11.16,"(40.68061065673828, -73.92864227294923)","(40.69804382324219, -73.92427825927734)"
5005542,1,3.56,01/01/2016 12:19:39 AM,01/01/2016 12:39:18 AM,16.80,"(40.72317504882813, -73.95267486572266)","(40.76137924194336, -73.92391967773438)"
5005543,1,3.79,01/01/2016 12:19:33 AM,01/01/2016 12:39:48 AM,22.25,"(40.67610549926758, -73.97161102294923)","(40.64607238769531, -74.0131607055664)"
5005544,1,3.01,01/01/2016 12:22:12 AM,01/01/2016 12:38:32 AM,14.80,"(40.669578552246094, -73.989501953125)","(40.68903350830078, -74.00064849853516)"
5005545,1,2.55,01/01/2016 12:24:01 AM,01/01/2016 12:39:22 AM,13.30,"(40.68285369873047, -73.96472930908203)","(40.66301345825195, -73.94071960449219)"
5005546,1,1.37,01/01/2016 12:32:59 AM,01/01/2016 12:39:35 AM,8.30,"(40.74645614624024, -73.89114379882812)","(40.74211120605469, -73.86774444580078)"
5005547,1,0.57,01/01/2016 12:34:42 AM,01/01/2016 12:39:21 AM,6.30,"(40.74619674682617, -73.89667510986328)","(40.745689392089844, -73.88619232177734)"
5005548,1,1.01,01/01/2016 12:31:23 AM,01/01/2016 12:39:36 AM,8.30,"(40.803558349609375, -73.95335388183594)","(40.79412078857422, -73.94915008544923)"
5005549,1,2.46,01/01/2016 12:24:40 AM,01/01/2016 12:39:52 AM,15.30,"(40.70281600952149, -73.99406433105469)","(40.679725646972656, -73.97157287597656)"
5005550,1,1.61,01/01/2016 12:28:59 AM,01/01/2016 12:39:23 AM,11.90,"(40.75664138793945, -73.91413116455078)","(40.73965835571289, -73.91754913330078)"


In [35]:
#calc_min((40.75664138793945, -73.91413116455078), oepnv_locations)
t0 = time.time()
taxi_df_stripped.head(100)['trip_start_coords'].map(calc_min)
t1 = time.time()
total = t1 - t0
total

2.792691946029663

In [33]:
30 * 9018 / 60 / 60

75.15

In [25]:
def calc_min(x):
    amin = np.argmin([np.linalg.norm(np.array(x)-np.array(y)) for y in oepnv_locations['coords']])
    return oepnv_locations.iloc[amin]['location']